<a href="https://colab.research.google.com/github/bhardwajshivam/Binomial-Option-Pricing-Model/blob/main/Binomial_Option_Pricing_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from functools import wraps
from time import time

In [2]:
def timing(f):
  @wraps(f)
  def wrap(*args,**kw):
    ts = time()
    result = f(*args,**kw)
    te = time()
    print('func:%r args:[%r,%r] took: %2.4f sec'% \
          (f.__name__, args, kw, te-ts))

    return result
  return wrap

In [3]:
#Binomial tree

# Initialise parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.06      # annual risk-free rate
N = 3         # number of time steps
u = 1.1       # up-factor in binomial models
d = 1/u       # ensure recombining tree
opttype = 'C' # Option Type 'C' or 'P'


In [8]:
@timing
def binomial_tree(K, T, S0, r, N, u, d, opttype = 'C'):
  #pre computing the constants
  dt = T/N
  q = (np.exp(r*dt) - d) / (u - d)
  disc = np.exp(-r*dt)

  #initialise asset prices at maturity - Time step N
  S = np.zeros(N+1)
  S[0] = S0*d**N
  for j in range(1, N+1):
    S[j] = S[j-1]*u/d

  #initialising option values at maturity
  C = np.zeros(N+1)
  for j in range(0,N+1):
    C[j] = max(0, S[j]-K)

  #Iterating backwards to get fair price of the option at t=0 / today
  for i in np.arange(N, 0, -1):
    for j in range(0,i):
      C[j] = disc*(q*C[j+1] + (1-q)*C[j])

  return C[0]

binomial_tree(K,T,S0,r,N,u,d,opttype='C')

func:'binomial_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0015 sec


10.145735799928817

In [9]:
#Optimising

@timing
def binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C'):
    #precompute constants
    dt = T/N
    q = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)

    # initialise asset prices at maturity - Time step N
    C = S0 * d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1))

    # initialise option values at maturity
    C = np.maximum( C - K , np.zeros(N+1) )

    # step backwards through tree
    for i in np.arange(N,0,-1):
        C = disc * ( q * C[1:i+1] + (1-q) * C[0:i] )

    return C[0]

binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C')

func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0011 sec


10.145735799928826

In [10]:
#comparing the two approaches

for N in [3,50, 100, 1000, 5000]:
    binomial_tree(K,T,S0,r,N,u,d,opttype='C')
    binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C')

func:'binomial_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0001 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0001 sec
func:'binomial_tree' args:[(100, 1, 100, 0.06, 50, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0017 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 50, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0007 sec
func:'binomial_tree' args:[(100, 1, 100, 0.06, 100, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0064 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 100, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0015 sec
func:'binomial_tree' args:[(100, 1, 100, 0.06, 1000, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.9774 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 1000, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0127 sec
func:'binomial_tree' args:[(100, 1, 100, 0.06, 5000, 1.1, 0.9090909090909091),{'opttype'

# Barrier options with binomial trees

Barrier Options with [the](https://) Binomial Asset Pricing Model

Implementation of a simple slow and fast barrier tree pricing model in python. We will treat binomial tree as a network with nodes (i,j) with i representing the time steps and j representing the number of ordered price outcome (lowest - or bottom of tree - to highest).

In [11]:

# Initialise parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
H = 125       # up-and-out barrier price/value
r = 0.06      # annual risk-free rate
N = 3         # number of time steps
u = 1.1       # up-factor in binomial models
d = 1/u       # ensure recombining tree
opttype = 'C' # Option Type 'C' or 'P'

In [12]:
@timing
def barrier_tree_fast(K,T,S0,H,r,N,u,d,opttype='C'):
    #precompute values
    dt = T/N
    q = (np.exp(r*dt) - d)/(u-d)
    disc = np.exp(-r*dt)

    # initialise asset prices at maturity
    S = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

    # option payoff
    if opttype == 'C':
        C = np.maximum( S - K, 0 )
    else:
        C = np.maximum( K - S, 0 )

    # check terminal condition payoff
    C[S >= H] = 0

    # backward recursion through the tree
    for i in np.arange(N-1,-1,-1):
        S = S0 * d**(np.arange(i,-1,-1)) * u**(np.arange(0,i+1,1))
        C[:i+1] = disc * ( q * C[1:i+2] + (1-q) * C[0:i+1] )
        C = C[:-1]
        C[S >= H] = 0
    return C[0]

barrier_tree_fast(K,T,S0,H,r,N,u,d,opttype='C')

func:'barrier_tree_fast' args:[(100, 1, 100, 125, 0.06, 3, 1.1, 0.9090909090909091),{'opttype': 'C'}] took: 0.0028 sec


4.00026736854323

# American Put Options with the Binomial Asset Pricing Model


Implementation of a simple slow and fast binomial pricing model in python. We will treat binomial tree as a network with nodes (i,j) with i representing the time steps and j representing the number of ordered price outcome (lowest - or bottom of tree - to highest).

In [13]:
@timing
def american_fast_tree(K,T,S0,r,N,u,d,opttype='P'):
    #precompute values
    dt = T/N
    q = (np.exp(r*dt) - d)/(u-d)
    disc = np.exp(-r*dt)

    # initialise stock prices at maturity
    S = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

    # option payoff
    if opttype == 'P':
        C = np.maximum(0, K - S)
    else:
        C = np.maximum(0, S - K)

    # backward recursion through the tree
    for i in np.arange(N-1,-1,-1):
        S = S0 * d**(np.arange(i,-1,-1)) * u**(np.arange(0,i+1,1))
        C[:i+1] = disc * ( q*C[1:i+2] + (1-q)*C[0:i+1] )
        C = C[:-1]
        if opttype == 'P':
            C = np.maximum(C, K - S)
        else:
            C = np.maximum(C, S - K)

    return C[0]

american_fast_tree(K,T,S0,r,N,u,d,opttype='P')

func:'american_fast_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091),{'opttype': 'P'}] took: 0.0018 sec


4.654588754602527